# Capstone Project - The Battle of Neighborhoods

## Suitable New Store Locations in Paris for a restaurent


### introduction 

Paris is the capital and most populous city of France. Since the 17th century, Paris has been one of Europe's major centres of finance, diplomacy, commerce, fashion, science and arts. 
Paris received 24.5 million visitors in 2018, measured by hotel stays, with the largest numbers of foreign visitors coming from the United States, the United Kingdom, Germany and China. It was ranked as the second most visited travel destination in the world in 2018, after Bangkok.

For the Capstone project, I chose the city of Paris to analyze its geospatial data in order to select a location for a new restaurant. The decision will have to be made using a data-driven approach, where we will analyze the geospatial data and use the location of existing restaurants, cafés and bistrots.



### Data Research and Preparation

#### Import the Paris District Data

Paris is divided into 20 Municipal Districts (or administrative districts), reduced to arrondissements. They are normally referred to by the district number rather than by a name.

Data of the arrondissements is necessary to select the most suitable of our restaurant.

The data used is available on the web and can be manipulated and cleaned to provide a meaningful dataset to use.
the data used is from : https://www.data.gouv.fr/fr/datasets/arrondissements-1/
after importing the data we can croceed to cleaning of the data set

In [6]:
# Import libraries
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import pandas as pd

import folium 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

'''!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup

# Import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')'''

"!conda install -c conda-forge geopy --yes \nfrom geopy.geocoders import Nominatim # convert an address into latitude and longitude values\n\nimport requests # library to handle requests\nfrom pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe\n\n# Matplotlib and associated plotting modules\nimport matplotlib.pyplot as plt\nimport matplotlib.cm as cm\nimport matplotlib.colors as colors\n\nfrom bs4 import BeautifulSoup\n\n# Import k-means from clustering stage\nfrom sklearn.cluster import KMeans\n\n!conda install -c conda-forge folium=0.5.0 --yes \nimport folium # map rendering library\n\nprint('Libraries imported.')"

In [2]:
paris = pd.read_csv("/Users/ahmedjabrane/downloads/arrondissements.csv", sep='\;')
lat_split = []
long_split = []
for i in range(paris.shape[0]):
    g = paris['Geometry X Y'].iloc[i].split(',')
    lat_split.append(g[0])
    long_split.append(g[1])
    
paris['lat'] = lat_split
paris['lon'] = long_split

paris.head()

/Users/ahmedjabrane/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,N_SQ_AR,C_AR,C_ARINSEE,L_AR,L_AROFF,N_SQ_CO,SURFACE,PERIMETRE,Geometry X Y,Geometry,OBJECTID,LONGUEUR,lat,lon
0,750000001,1,75101,1er Ardt,Louvre,750001537,1.824613e+06,6054.936862,"48.8625627018,2.33644336205","""{""""type"""": """"Polygon"""", """"coordinates"""": [[[2...",1,6054.680862,48.8625627018,2.33644336205
1,750000002,2,75102,2ème Ardt,Bourse,750001537,9.911537e+05,4554.104360,"48.8682792225,2.34280254689","""{""""type"""": """"Polygon"""", """"coordinates"""": [[[2...",2,4553.938764,48.8682792225,2.34280254689
2,750000017,17,75117,17ème Ardt,Batignolles-Monceau,750001537,5.668835e+06,10775.579516,"48.887326522,2.30677699057","""{""""type"""": """"Polygon"""", """"coordinates"""": [[[2...",17,10775.348915,48.887326522,2.30677699057
3,750000014,14,75114,14ème Ardt,Observatoire,750001537,5.614877e+06,10317.483310,"48.8292445005,2.3265420442","""{""""type"""": """"Polygon"""", """"coordinates"""": [[[2...",14,10316.720258,48.8292445005,2.3265420442
4,750000020,20,75120,20ème Ardt,Ménilmontant,750001537,5.983446e+06,10704.940486,"48.8634605789,2.40118812928","""{""""type"""": """"Polygon"""", """"coordinates"""": [[[2...",20,10704.483774,48.8634605789,2.40118812928


#### Exploring, Wrangling and Cleaning the Data¶

In [3]:
paris = paris.drop(['OBJECTID','N_SQ_AR','Geometry X Y','C_ARINSEE','N_SQ_CO','SURFACE','Geometry','LONGUEUR','PERIMETRE'], axis=1)
paris.rename(columns={'C_AR': 'Arrondissement_Num','L_AROFF': 'Neighborhood ', 'L_AR': 'french name', 'Geometry_X': 'Latitude', 'Geometry_Y': 'Longitude',  'LAR': 'French_Name'}, inplace=True)
paris

,Arrondissement_Num,french name,Neighborhood,lat,lon
0,1,1er Ardt,Louvre,48.8625627018,2.33644336205
1,2,2ème Ardt,Bourse,48.8682792225,2.34280254689
2,17,17ème Ardt,Batignolles-Monceau,48.887326522,2.30677699057
3,14,14ème Ardt,Observatoire,48.8292445005,2.3265420442
4,20,20ème Ardt,Ménilmontant,48.8634605789,2.40118812928
5,7,7ème Ardt,Palais-Bourbon,48.8561744288,2.31218769148
6,11,11ème Ardt,Popincourt,48.8590592213,2.3800583082
7,13,13ème Ardt,Gobelins,48.8283880317,2.36227244042
8,4,4ème Ardt,Hôtel-de-Ville,48.8543414263,2.35762962032
9,8,8ème Ardt,Élysée,48.8727208374,2.3125540224


#### the latitude and longitude values of Paris

In [4]:
from geopy.geocoders import Nominatim 

address = 'Paris'

# Define the user_agent as Paris_explorer
geolocator = Nominatim(user_agent="Paris_explorer")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinates of Paris France are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Paris France are 48.8566969, 2.3514616.


#### Create a map of Paris with arrondissements

In [9]:
# create map of Paris using the above latitude and longitude values
map_paris = folium.Map(location=[latitude, longitude], zoom_start=12)


# add markers to map
for lat, lng, label in zip(paris['lat'], paris['lon'], paris['french name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=25,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_paris)  
    
map_paris

#### Week 1
We now have located and imported the relevant data for the districts of Paris.
Our business objective, strategy and methods to achieve our goal have been laid out, and a data workflow established.
Next up, we will leverage Foursquare location data to obtain data on high traffic areas - where consumers go for shopping, restaurants and entertainment - in all of the 20 districts.
The Battle of Neighborhoods continues in the next section.